In [3]:
# # Import librarys

import pickle
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

# from sklearn.metrics import roc_curve, roc_auc_score
# from scipy.interpolate import interp1d

# import plotly.graph_objects as go

from ipywidgets import widgets, Layout, Label, HBox, VBox, Box
from IPython.display import clear_output, display, Markdown

In [4]:
# Importamos los modelos
with open('./Models_app/clf_reAMI_6.pickle', 'rb') as f:
    clf_reAMI_6 = pickle.load(f)
with open('./Models_app/clf_reAMI_12.pickle', 'rb') as f:
    clf_reAMI_12 = pickle.load(f)
with open('./Models_app/clf_reAMI_24.pickle', 'rb') as f:
    clf_reAMI_24 = pickle.load(f)
with open('./Models_app/clf_bleed_6.pickle', 'rb') as f:
    clf_bleed_6 = pickle.load(f)
with open('./Models_app/clf_bleed_12.pickle', 'rb') as f:
    clf_bleed_12 = pickle.load(f)
with open('./Models_app/clf_bleed_24.pickle', 'rb') as f:
    clf_bleed_24 = pickle.load(f)

In [5]:
style = {'description_width': 'initial'}

output_all = widgets.Output()
output_classification = widgets.Output()


def run_all():
    
    ######################
    # DATA INPUT WIDGETS #
    ######################
    Hemoglobin_Admission = widgets.FloatSlider(value=14, min=0, max=30.0, step=0.05,
                                               description= "Hemoglobina al ingreso (g/dl)", disabled = False,
                                               continuous_update=True, orientation='horizontal',
                                               readout=True,readout_format='.1f', style = style,
                                               layout = Layout(width = '475px'))

    PeripheralArteryDisease = widgets.Checkbox(value=False, continuous_update = True,
                                             description= "Enfermedad vascular periferica",
                                            disabled=False, indent=False, layout = Layout(width = '230px'))

    Revascularization_Complete = widgets.Checkbox(value=False, continuous_update = True,
                                             description= "Revascularización completa",
                                            disabled=False, indent=False, layout = Layout(width = '200px'))

    Killip2orMore = widgets.Checkbox(value=False, continuous_update = True,
                                             description= "Killip 2 o más",
                                            disabled=False, indent=False, layout = Layout(width = '200px'))

    Age = widgets.IntSlider(value=60, min=0, max=120.0, step=1, 
                              description= "Edad", disabled = False, 
                              continuous_update=True, orientation='horizontal', readout=True,readout_format='.0f', 
                              style = style, layout = Layout(width = '475px'))

    Creatinine_Admission = widgets.FloatSlider(value=0.99, min=0, max=20, step=0.01, 
                                               description= "Creatinina al ingreso (mg/dl)", disabled = False, 
                                               continuous_update=True, orientation='horizontal', 
                                               readout=True,readout_format='.2f',
                                               style = style, layout = Layout(width = '475px'))

    Bleeding_inhospital = widgets.Checkbox(value=False, continuous_update = True,
                                             description= "Sangrado en el hospital",
                                            disabled=False, indent=False, layout = Layout(width = '200px'))

    Multivessel = widgets.Checkbox(value=False, continuous_update = True,
                                             description= "Enfermedad multivaso",
                                            disabled=False, indent=False, layout = Layout(width = '200px'))

    DrugElutingStent = widgets.Checkbox(value=False, continuous_update = True,
                                             description= "Stent farmacologico",
                                            disabled=False, indent=False, layout = Layout(width = '200px'))

    ReAMI_inhospital = widgets.Checkbox(value=False, continuous_update = True,
                                             description= "Reinfarto en el hospital",
                                            disabled=False, indent=False, layout = Layout(width = '200px'))
    
     
    ######################
    ######  OUTPUT  ######
    ######################
    
    def get_prob(X, model_1, model_2):
        prob_1 = model_1.predict_proba(X)[:,1]
        prob_2 = model_2.predict_proba(X)[:,1]
        prob = (prob_2 - prob_1) / (1 - prob_1)

        return abs(prob[0])

    def get_recomen(X, clf_reAMI_6, clf_reAMI_12, clf_reAMI_24, clf_bleed_6, clf_bleed_12, clf_bleed_24):
        prob_reAMI_6_12 = get_prob(X, clf_reAMI_6, clf_reAMI_12)
        prob_reAMI_12_24 = get_prob(X, clf_reAMI_12, clf_reAMI_24)

        prob_bleed_6_12 = get_prob(X, clf_bleed_6, clf_bleed_12)
        prob_bleed_12_24 = get_prob(X, clf_bleed_12, clf_bleed_24)
        
        recomendacion_cabecera = '### Recomendación'
        display(Markdown(recomendacion_cabecera))
        
        if (prob_reAMI_12_24 > prob_bleed_12_24):
            recomendacion = 'Se recomienda tomar la medicación Doble Antiagregación durante 24 meses.'
            return display(Markdown(recomendacion))
        elif (prob_reAMI_6_12 > prob_bleed_6_12):
            recomendacion = 'Se recomienda tomar la medicación Doble Antiagregación durante 12 meses.'
            return display(Markdown(recomendacion))
        else:
            recomendacion = 'Se recomienda tomar la medicación Doble Antiagregación durante 6 meses.'
            return display(Markdown(recomendacion))
    
    def callback(w):
        with output:
            try:
                clear_output()
                X = pd.DataFrame({"Hemoglobin_Admission": [Hemoglobin_Admission.value], 
                                "PeripheralArteryDisease": [PeripheralArteryDisease.value],
                                "Revascularization_Complete": [Revascularization_Complete.value],
                                "Killip2orMore": [Killip2orMore.value],
                                "Age": [Age.value],
                                "Creatinine_Admission": [Creatinine_Admission.value],
                                "Bleeding_inhospital": [Bleeding_inhospital.value],
                                "Multivessel": [Multivessel.value],
                                "DrugElutingStent": [DrugElutingStent.value],
                                "ReAMI_inhospital": [ReAMI_inhospital.value]})
                
                tabla = f'|   	| 6-12 meses  	| 12-24 meses  	|\n\
                |:-:	|:-:	|:-:	|\n\
                |**Reinfarto** 	|{100*get_prob(X, clf_reAMI_6, clf_reAMI_12):.2f}%	|{100*get_prob(X, clf_reAMI_12, clf_reAMI_24):.2f}%	|\n\
                |**Hemorragia**	|{100*get_prob(X, clf_bleed_6, clf_bleed_12):.2f}%	|{100*get_prob(X, clf_bleed_12, clf_bleed_24):.2f}%	|\n\n'
            
                nombre_tabla = '### Riesgo estimado de infarto y hemorragia en los periodos establecidos si \
                se prosigue la terapia de doble antiagregación'
                
                display(Markdown(nombre_tabla))
                display(Markdown(tabla))
                
                get_recomen(X, clf_reAMI_6, clf_reAMI_12, clf_reAMI_24, clf_bleed_6, clf_bleed_12, clf_bleed_24)
            except:
                pass
        
    
    
    
    ######################
    ####  HEAD BOARD  ####
    ######################
    cabecera_text = '# Riesgo de Reinfarto / Hemorragia \n'
    
    cabecera2_text = 'Aplicación web del riesgo de sufrir un infarto o hemorragia a los \
                    6-12 y 12-24 meses para pacientes que hayan sufrido un infarto agudo de \
                    miocardio. El código empleado para el entrenamiento y validación de los modelos \
                    se encuentra en <https://github.com/PabloPerSa/TFG>.'
    cabecera = Markdown(cabecera_text)
    cabecera2 = Markdown(cabecera2_text)
    output_cabecera = widgets.Output()
    
    with output_cabecera:
        display(cabecera)
        display(cabecera2)
    
    cabecera_final = HBox([output_cabecera], layout = Layout(width = '1450px', justify_content = 'space-between'))
    
    ######################
    ######  FOOTER  ######
    ######################
    pie_text = '**AVISO: Esta calculadora se ha desarrollado como un PROTOTIPO para la demostración \
    de las posibilides de incluir Inteligencia Artificial en la prescripción de tratamientos post-infarto. \
    Esta calculadora está dirigida a científicos de datos y profesionales de la salud con interés en \
    desarrollar una herramienta de similares características y funcional. Ninguno de las resultados \
    mostrados se puede interpretar como consejo médico. Si usted ha padecido un infarto y está buscando \
    información sobre su tratamiento farmacológico, consulte con su cardiólogo o médico de cabecera.**'
    
    pie = Markdown(pie_text)
    output_footer = widgets.Output()
    
    with output_footer:
        display(pie)
    
    pie_final = HBox([output_footer], layout = Layout(width = '720px', justify_content = 'space-between'))
    
    #####################
    ######  VOILA  ######
    #####################

    submit = widgets.Button(description='Enviar')
    submit.on_click(callback)
    
    output = widgets.Output()
    
    tab2a = widgets.VBox(children=[Hemoglobin_Admission, PeripheralArteryDisease, Revascularization_Complete, 
                               Killip2orMore, Age, Creatinine_Admission, Bleeding_inhospital, Multivessel,
                               DrugElutingStent, ReAMI_inhospital],
                         layout = Layout(width = '520px', justify_content = 'flex-start'))
    
    DATA_BOX = widgets.VBox([tab2a, submit], layout = Layout(width = '520px', justify_content = 'flex-start'))
    
#     RES_BOX = widgets.VBox([output], layout = Layout(width = '720px'))
    RES_BOX = widgets.VBox([pie_final, output], layout = Layout(width = '720px'))

    TOTAL_BOX = HBox([DATA_BOX, RES_BOX], layout = Layout(width = '1450px', justify_content = 'flex-start'))
    
    global output_all
    with output_all:
        clear_output()
        display(cabecera_final)
        display(TOTAL_BOX)
#         display(pie_final)

In [6]:
run_all()
display(output_all)

Output()